In [1]:
import xarray
import numpy as np
import zarr
from dask.diagnostics import ProgressBar
from dask.distributed import Client, LocalCluster, progress
from rechunker import rechunk
from tqdm import tqdm
import gc
import os

In [5]:
from dask.distributed import Client
client = Client(n_workers=12)

/scratch/snx3000/kschuurm/lightning-env/lib/python3.9/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 35483 instead
  warnings.warn(


In [3]:
import shutil
import gc

def rechunk_zarr_store(zarr, chunks, target_zarr, steps = 200, 
                       slice_lengths=None, iterate_over='time',
                      chunk_subset=None):
    
    if slice_lengths is None:
        slice_lengths = int(len(zarr[iterate_over]) / steps)
    if os.path.exists(target_zarr):
        temp = xarray.open_zarr(target_zarr)
        already_rechunked = len(temp[iterate_over])
        temp.close()
    else:
        already_rechunked = 0
    
    # already_rechunked =0
    for i in tqdm(range(0, len(zarr[iterate_over]), slice_lengths)):
        if i + slice_lengths <= already_rechunked:
            print(f'skipped {i}')
            continue

        if i < already_rechunked and i + slice_lengths > already_rechunked:
            zarr_slice = zarr.isel({iterate_over : slice(already_rechunked, i+slice_lengths)})
        else:
            zarr_slice = zarr.isel({iterate_over : slice(i, i + slice_lengths)})
        
        for v in list(zarr_slice.coords.keys()):
            if zarr_slice.coords[v].dtype == object:
                zarr_slice[v].encoding.clear()
                
        for var in zarr_slice:
            zarr_slice[var].encoding.clear()
        
        if chunk_subset is not None:
            zarr_slice[chunk_subset] = zarr_slice[chunk_subset].chunk(chunks)
        else:
            zarr_slice = zarr_slice.chunk(chunks)
            
        if i == 0:
            zarr_slice.to_zarr(target_zarr, mode="w")
        else:
            zarr_slice.to_zarr(target_zarr, append_dim=iterate_over)
        del zarr_slice
        gc.collect()

In [8]:
seviri18 = xarray.open_zarr('/scratch/snx3000/acarpent/SEVIRI_2018.zarr')


In [10]:
seviri18['time'] = seviri18.time.copy()
seviri18 = seviri18.transpose('data', 'time', 'y','x')
seviri18[['data','time','y','x', '__xarray_dataarray_variable__']]

<xarray.Dataset>
Dimensions:                        (data: 11, time: 29757, y: 658, x: 736)
Coordinates:
  * data                           (data) object 'IR_016' 'IR_039' ... 'WV_073'
  * time                           (time) datetime64[ns] 2018-01-01T00:15:00 ...
  * y                              (y) float64 28.98 29.03 29.08 ... 61.78 61.83
  * x                              (x) float64 -7.775 -7.725 ... 28.93 28.98
Data variables:
    __xarray_dataarray_variable__  (data, time, y, x) float32 dask.array<chunksize=(11, 1, 658, 736), meta=np.ndarray>

In [11]:
target_zarr = '/scratch/snx3000/kschuurm/ZARR/SEVIRI_2018.zarr'

chunks = {'data':-1, 'time':1, 'y':-1, 'x':-1}

rechunk_zarr_store(seviri18[['data','time','y','x', '__xarray_dataarray_variable__']], chunks, target_zarr, slice_lengths=1000, 
                   iterate_over='time', chunk_subset='__xarray_dataarray_variable__')

  0%|          | 0/30 [00:00<?, ?it/s]

skipped 0
skipped 1000
skipped 2000
skipped 3000
skipped 4000
skipped 5000
skipped 6000
skipped 7000
skipped 8000
skipped 9000
skipped 10000
skipped 11000
skipped 12000
skipped 13000
skipped 14000
skipped 15000
skipped 16000
skipped 17000
skipped 18000
skipped 19000
skipped 20000
skipped 21000
skipped 22000
skipped 23000
skipped 24000
skipped 25000
skipped 26000
skipped 27000
skipped 28000


 97%|█████████▋| 29/30 [00:01<00:00, 22.98it/s]


ValueError: parameter 'value': expected array with shape (29757, 0, 658, 736), got (11, 0, 658, 736)

In [6]:
test = = xarray.open_zarr('/scratch/snx3000/kschuurm/ZARR/SEVIRI_2018.zarr')

In [7]:
sarah.transpose('data', 'time', 'y','x')

<xarray.Dataset>
Dimensions:                        (time: 29757, data: 11, y: 658, x: 736)
Coordinates:
  * data                           (data) object 'IR_016' 'IR_039' ... 'WV_073'
  * time                           (time) datetime64[ns] 2018-01-01T00:15:00 ...
  * x                              (x) float64 -7.775 -7.725 ... 28.93 28.98
  * y                              (y) float64 28.98 29.03 29.08 ... 61.78 61.83
Data variables:
    __xarray_dataarray_variable__  (data, time, y, x) float32 dask.array<chunksize=(11, 1, 658, 736), meta=np.ndarray>

In [14]:
xarray.open_zarr('/scratch/snx3000/acarpent/SEVIRI_2016-2022.zarr')

<xarray.Dataset>
Dimensions:                        (data: 11, time: 161712, y: 658, x: 736)
Coordinates:
  * data                           (data) object 'IR_016' 'IR_039' ... 'WV_073'
  * time                           (time) datetime64[ns] 2016-01-01 ... 2022-...
  * x                              (x) float64 -7.775 -7.725 ... 28.93 28.98
  * y                              (y) float64 28.98 29.03 29.08 ... 61.78 61.83
Data variables:
    __xarray_dataarray_variable__  (data, time, y, x) float16 dask.array<chunksize=(11, 1, 658, 736), meta=np.ndarray>